# Load data

In [1]:
import pandas as pd
from sklearn.model_selection import cross_validate

In [2]:
test = pd.read_csv('test.csv')
test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,A,B,A,C,B,D,A,E,E,...,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
1,5,A,B,A,C,B,D,A,E,C,...,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
2,15,A,B,A,C,B,D,A,E,C,...,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
3,16,A,A,B,A,B,D,A,E,E,...,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
4,17,A,B,A,A,B,B,A,E,E,...,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931


In [3]:
train = pd.read_csv('train.csv')
train.head()
train = train.sample(frac=1, random_state=42)

In [4]:
cat_cols = [x for x in train.columns if x.startswith('cat')]
cat_cols

['cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9']

In [5]:
x_train = train.drop(columns=['target','id'])
y_train = train['target']

In [6]:
for cat in cat_cols:
    x_train[cat] = x_train[cat].astype("category")
    test[cat] = test[cat].astype("category")

# Submit

In [7]:
def submit(model):
    model.fit(x_train, y_train)
    pred = model.predict(test.drop(columns='id'))
    submission = pd.DataFrame({'id':test['id'], 'target':pred})
    return submission

# Train a lightgbm boosted tree (default parameters)

In [13]:
from lightgbm.sklearn import LGBMRegressor

In [14]:
bt = LGBMRegressor(n_jobs=-2)

In [15]:
scores = cross_validate(bt, X=x_train, y=y_train, cv=5, return_train_score = True,
                         scoring='neg_root_mean_squared_error')

In [16]:
scores

{'fit_time': array([1.22706318, 1.18083525, 1.15409446, 1.17929196, 1.16860175]),
 'score_time': array([0.08760285, 0.09076786, 0.09165931, 0.09075093, 0.0900774 ]),
 'test_score': array([-0.84593716, -0.84811234, -0.84677842, -0.84532452, -0.84509775]),
 'train_score': array([-0.83678672, -0.83618969, -0.83649384, -0.83679032, -0.83688036])}

In [17]:
scores['test_score'].mean()

-0.8462500379112754

In [18]:
submission = submit(bt)
submission.head()

,id,target
0,0,7.621181
1,5,7.865160
2,15,7.602939
3,16,7.441470
4,17,7.256050


In [14]:
submission.to_csv('submission_lightgbm_default.csv', index=False)

# Train a catboost model (default parameters)

In [8]:
from catboost import CatBoostRegressor

In [9]:
cbr = CatBoostRegressor(cat_features=cat_cols, thread_count=6)

In [10]:
scores = cross_validate(cbr, X=x_train, y=y_train, cv=5, return_train_score = True,                        
                         scoring='neg_root_mean_squared_error')

Learning rate set to 0.103533
0:	learn: 0.8837729	total: 191ms	remaining: 3m 10s
1:	learn: 0.8811223	total: 269ms	remaining: 2m 14s
2:	learn: 0.8788712	total: 383ms	remaining: 2m 7s
3:	learn: 0.8769321	total: 452ms	remaining: 1m 52s
4:	learn: 0.8753202	total: 534ms	remaining: 1m 46s
5:	learn: 0.8736118	total: 649ms	remaining: 1m 47s
6:	learn: 0.8722263	total: 736ms	remaining: 1m 44s
7:	learn: 0.8710068	total: 824ms	remaining: 1m 42s
8:	learn: 0.8697825	total: 918ms	remaining: 1m 41s
9:	learn: 0.8687360	total: 1.01s	remaining: 1m 39s
10:	learn: 0.8677390	total: 1.09s	remaining: 1m 37s
11:	learn: 0.8669163	total: 1.19s	remaining: 1m 37s
12:	learn: 0.8661553	total: 1.27s	remaining: 1m 36s
13:	learn: 0.8653393	total: 1.36s	remaining: 1m 36s
14:	learn: 0.8645831	total: 1.45s	remaining: 1m 35s
15:	learn: 0.8639242	total: 1.53s	remaining: 1m 34s
16:	learn: 0.8633693	total: 1.6s	remaining: 1m 32s
17:	learn: 0.8628391	total: 1.68s	remaining: 1m 31s
18:	learn: 0.8622867	total: 1.79s	remaining: 1

157:	learn: 0.8444963	total: 15.8s	remaining: 1m 24s
158:	learn: 0.8444437	total: 15.9s	remaining: 1m 24s
159:	learn: 0.8443628	total: 16s	remaining: 1m 24s
160:	learn: 0.8442982	total: 16.1s	remaining: 1m 24s
161:	learn: 0.8442421	total: 16.2s	remaining: 1m 24s
162:	learn: 0.8441735	total: 16.4s	remaining: 1m 23s
163:	learn: 0.8441079	total: 16.5s	remaining: 1m 23s
164:	learn: 0.8440362	total: 16.6s	remaining: 1m 23s
165:	learn: 0.8439763	total: 16.7s	remaining: 1m 23s
166:	learn: 0.8439109	total: 16.8s	remaining: 1m 23s
167:	learn: 0.8438452	total: 16.9s	remaining: 1m 23s
168:	learn: 0.8437931	total: 17s	remaining: 1m 23s
169:	learn: 0.8437318	total: 17.1s	remaining: 1m 23s
170:	learn: 0.8436771	total: 17.2s	remaining: 1m 23s
171:	learn: 0.8436161	total: 17.3s	remaining: 1m 23s
172:	learn: 0.8435546	total: 17.4s	remaining: 1m 23s
173:	learn: 0.8435003	total: 17.5s	remaining: 1m 23s
174:	learn: 0.8434542	total: 17.6s	remaining: 1m 22s
175:	learn: 0.8433718	total: 17.7s	remaining: 1m 2

313:	learn: 0.8380491	total: 31.8s	remaining: 1m 9s
314:	learn: 0.8380365	total: 31.9s	remaining: 1m 9s
315:	learn: 0.8379803	total: 32s	remaining: 1m 9s
316:	learn: 0.8379555	total: 32.1s	remaining: 1m 9s
317:	learn: 0.8379276	total: 32.2s	remaining: 1m 9s
318:	learn: 0.8378890	total: 32.3s	remaining: 1m 8s
319:	learn: 0.8378573	total: 32.4s	remaining: 1m 8s
320:	learn: 0.8378377	total: 32.5s	remaining: 1m 8s
321:	learn: 0.8378323	total: 32.6s	remaining: 1m 8s
322:	learn: 0.8377819	total: 32.7s	remaining: 1m 8s
323:	learn: 0.8377472	total: 32.8s	remaining: 1m 8s
324:	learn: 0.8377104	total: 32.9s	remaining: 1m 8s
325:	learn: 0.8376846	total: 33s	remaining: 1m 8s
326:	learn: 0.8376537	total: 33.1s	remaining: 1m 8s
327:	learn: 0.8376155	total: 33.2s	remaining: 1m 8s
328:	learn: 0.8375806	total: 33.3s	remaining: 1m 7s
329:	learn: 0.8375723	total: 33.4s	remaining: 1m 7s
330:	learn: 0.8375310	total: 33.5s	remaining: 1m 7s
331:	learn: 0.8374991	total: 33.6s	remaining: 1m 7s
332:	learn: 0.83

474:	learn: 0.8332626	total: 48.2s	remaining: 53.3s
475:	learn: 0.8332248	total: 48.3s	remaining: 53.1s
476:	learn: 0.8332047	total: 48.4s	remaining: 53.1s
477:	learn: 0.8331711	total: 48.5s	remaining: 53s
478:	learn: 0.8331438	total: 48.6s	remaining: 52.8s
479:	learn: 0.8331111	total: 48.7s	remaining: 52.7s
480:	learn: 0.8330923	total: 48.8s	remaining: 52.6s
481:	learn: 0.8330521	total: 48.9s	remaining: 52.5s
482:	learn: 0.8330100	total: 49s	remaining: 52.4s
483:	learn: 0.8329956	total: 49.1s	remaining: 52.3s
484:	learn: 0.8329653	total: 49.2s	remaining: 52.2s
485:	learn: 0.8329476	total: 49.3s	remaining: 52.1s
486:	learn: 0.8329294	total: 49.4s	remaining: 52s
487:	learn: 0.8329200	total: 49.5s	remaining: 51.9s
488:	learn: 0.8328845	total: 49.6s	remaining: 51.8s
489:	learn: 0.8328507	total: 49.7s	remaining: 51.7s
490:	learn: 0.8328335	total: 49.8s	remaining: 51.6s
491:	learn: 0.8328108	total: 49.9s	remaining: 51.5s
492:	learn: 0.8327781	total: 50s	remaining: 51.5s
493:	learn: 0.832750

636:	learn: 0.8293971	total: 1m 4s	remaining: 36.6s
637:	learn: 0.8293688	total: 1m 4s	remaining: 36.5s
638:	learn: 0.8293518	total: 1m 4s	remaining: 36.4s
639:	learn: 0.8293282	total: 1m 4s	remaining: 36.3s
640:	learn: 0.8293025	total: 1m 4s	remaining: 36.3s
641:	learn: 0.8292644	total: 1m 4s	remaining: 36.2s
642:	learn: 0.8292266	total: 1m 4s	remaining: 36.1s
643:	learn: 0.8292024	total: 1m 5s	remaining: 36s
644:	learn: 0.8291926	total: 1m 5s	remaining: 35.9s
645:	learn: 0.8291812	total: 1m 5s	remaining: 35.8s
646:	learn: 0.8291633	total: 1m 5s	remaining: 35.7s
647:	learn: 0.8291292	total: 1m 5s	remaining: 35.6s
648:	learn: 0.8291057	total: 1m 5s	remaining: 35.5s
649:	learn: 0.8290700	total: 1m 5s	remaining: 35.4s
650:	learn: 0.8290420	total: 1m 5s	remaining: 35.3s
651:	learn: 0.8290116	total: 1m 5s	remaining: 35.2s
652:	learn: 0.8289841	total: 1m 6s	remaining: 35.1s
653:	learn: 0.8289796	total: 1m 6s	remaining: 35s
654:	learn: 0.8289486	total: 1m 6s	remaining: 34.9s
655:	learn: 0.82

...

267:	learn: 0.8397027	total: 27.8s	remaining: 1m 15s
268:	learn: 0.8396456	total: 27.9s	remaining: 1m 15s
269:	learn: 0.8396242	total: 28s	remaining: 1m 15s
270:	learn: 0.8395898	total: 28.1s	remaining: 1m 15s
271:	learn: 0.8395479	total: 28.2s	remaining: 1m 15s
272:	learn: 0.8395082	total: 28.3s	remaining: 1m 15s
273:	learn: 0.8395056	total: 28.4s	remaining: 1m 15s
274:	learn: 0.8394883	total: 28.5s	remaining: 1m 15s
275:	learn: 0.8394523	total: 28.6s	remaining: 1m 15s
276:	learn: 0.8394258	total: 28.7s	remaining: 1m 14s
277:	learn: 0.8393838	total: 28.8s	remaining: 1m 14s
278:	learn: 0.8393441	total: 28.9s	remaining: 1m 14s
279:	learn: 0.8393088	total: 29s	remaining: 1m 14s
280:	learn: 0.8392715	total: 29.2s	remaining: 1m 14s
281:	learn: 0.8392124	total: 29.3s	remaining: 1m 14s
282:	learn: 0.8391772	total: 29.4s	remaining: 1m 14s
283:	learn: 0.8391694	total: 29.5s	remaining: 1m 14s
284:	learn: 0.8391422	total: 29.6s	remaining: 1m 14s
285:	learn: 0.8391035	total: 29.7s	remaining: 1m 1

425:	learn: 0.8349782	total: 44s	remaining: 59.4s
426:	learn: 0.8349610	total: 44.1s	remaining: 59.2s
427:	learn: 0.8349433	total: 44.2s	remaining: 59.1s
428:	learn: 0.8349219	total: 44.3s	remaining: 59s
429:	learn: 0.8348933	total: 44.4s	remaining: 58.9s
430:	learn: 0.8348815	total: 44.5s	remaining: 58.8s
431:	learn: 0.8348619	total: 44.6s	remaining: 58.7s
432:	learn: 0.8348605	total: 44.7s	remaining: 58.6s
433:	learn: 0.8348217	total: 44.8s	remaining: 58.4s
434:	learn: 0.8347881	total: 44.9s	remaining: 58.4s
435:	learn: 0.8347776	total: 45s	remaining: 58.2s
436:	learn: 0.8347585	total: 45.1s	remaining: 58.1s
437:	learn: 0.8347489	total: 45.2s	remaining: 58s
438:	learn: 0.8347290	total: 45.3s	remaining: 57.9s
439:	learn: 0.8346868	total: 45.4s	remaining: 57.8s
440:	learn: 0.8346511	total: 45.5s	remaining: 57.7s
441:	learn: 0.8346136	total: 45.6s	remaining: 57.6s
442:	learn: 0.8345934	total: 45.7s	remaining: 57.5s
443:	learn: 0.8345736	total: 45.8s	remaining: 57.4s
444:	learn: 0.834543

585:	learn: 0.8310313	total: 1m	remaining: 42.5s
586:	learn: 0.8310121	total: 1m	remaining: 42.3s
587:	learn: 0.8309930	total: 1m	remaining: 42.2s
588:	learn: 0.8309592	total: 1m	remaining: 42.1s
589:	learn: 0.8309284	total: 1m	remaining: 42s
590:	learn: 0.8309094	total: 1m	remaining: 41.9s
591:	learn: 0.8308782	total: 1m	remaining: 41.8s
592:	learn: 0.8308435	total: 1m	remaining: 41.7s
593:	learn: 0.8308225	total: 1m	remaining: 41.6s
594:	learn: 0.8307956	total: 1m	remaining: 41.5s
595:	learn: 0.8307908	total: 1m 1s	remaining: 41.4s
596:	learn: 0.8307539	total: 1m 1s	remaining: 41.3s
597:	learn: 0.8307274	total: 1m 1s	remaining: 41.2s
598:	learn: 0.8307045	total: 1m 1s	remaining: 41.1s
599:	learn: 0.8306822	total: 1m 1s	remaining: 41s
600:	learn: 0.8306609	total: 1m 1s	remaining: 40.9s
601:	learn: 0.8306155	total: 1m 1s	remaining: 40.8s
602:	learn: 0.8306038	total: 1m 1s	remaining: 40.7s
603:	learn: 0.8305906	total: 1m 1s	remaining: 40.6s
604:	learn: 0.8305544	total: 1m 2s	remaining: 

743:	learn: 0.8275485	total: 1m 15s	remaining: 26.1s
744:	learn: 0.8275432	total: 1m 16s	remaining: 26s
745:	learn: 0.8275371	total: 1m 16s	remaining: 25.9s
746:	learn: 0.8275082	total: 1m 16s	remaining: 25.8s
747:	learn: 0.8274827	total: 1m 16s	remaining: 25.7s
748:	learn: 0.8274724	total: 1m 16s	remaining: 25.6s
749:	learn: 0.8274414	total: 1m 16s	remaining: 25.5s
750:	learn: 0.8274405	total: 1m 16s	remaining: 25.4s
751:	learn: 0.8274272	total: 1m 16s	remaining: 25.3s
752:	learn: 0.8273974	total: 1m 16s	remaining: 25.2s
753:	learn: 0.8273682	total: 1m 16s	remaining: 25.1s
754:	learn: 0.8273399	total: 1m 17s	remaining: 25s
755:	learn: 0.8273266	total: 1m 17s	remaining: 24.9s
756:	learn: 0.8272956	total: 1m 17s	remaining: 24.8s
757:	learn: 0.8272677	total: 1m 17s	remaining: 24.7s
758:	learn: 0.8272394	total: 1m 17s	remaining: 24.6s
759:	learn: 0.8272070	total: 1m 17s	remaining: 24.5s
760:	learn: 0.8272059	total: 1m 17s	remaining: 24.4s
761:	learn: 0.8271790	total: 1m 17s	remaining: 24.

900:	learn: 0.8240758	total: 1m 31s	remaining: 10.1s
901:	learn: 0.8240573	total: 1m 32s	remaining: 10s
902:	learn: 0.8240284	total: 1m 32s	remaining: 9.9s
903:	learn: 0.8240007	total: 1m 32s	remaining: 9.8s
904:	learn: 0.8239724	total: 1m 32s	remaining: 9.7s
905:	learn: 0.8239672	total: 1m 32s	remaining: 9.59s
906:	learn: 0.8239297	total: 1m 32s	remaining: 9.49s
907:	learn: 0.8239031	total: 1m 32s	remaining: 9.39s
908:	learn: 0.8238668	total: 1m 32s	remaining: 9.29s
909:	learn: 0.8238532	total: 1m 32s	remaining: 9.19s
910:	learn: 0.8238310	total: 1m 32s	remaining: 9.08s
911:	learn: 0.8238242	total: 1m 33s	remaining: 8.98s
912:	learn: 0.8237948	total: 1m 33s	remaining: 8.88s
913:	learn: 0.8237713	total: 1m 33s	remaining: 8.78s
914:	learn: 0.8237459	total: 1m 33s	remaining: 8.68s
915:	learn: 0.8237310	total: 1m 33s	remaining: 8.57s
916:	learn: 0.8237197	total: 1m 33s	remaining: 8.47s
917:	learn: 0.8236902	total: 1m 33s	remaining: 8.37s
918:	learn: 0.8236671	total: 1m 33s	remaining: 8.27

In [11]:
scores

{'fit_time': array([102.28253484, 102.06701207, 102.38217974, 101.63458133,
        103.07627583]),
 'score_time': array([0.09103394, 0.09189773, 0.0910964 , 0.09150481, 0.09127855]),
 'test_score': array([-0.84341781, -0.84537946, -0.84437322, -0.84320546, -0.84200925]),
 'train_score': array([-0.82170936, -0.82121502, -0.82161349, -0.82272889, -0.82258591])}

In [12]:
scores['test_score'].mean()

-0.8436770380536011

In [19]:
submission = submit(cbr)
submission.head()

Learning rate set to 0.107728
0:	learn: 0.8839277	total: 146ms	remaining: 2m 25s
1:	learn: 0.8811116	total: 256ms	remaining: 2m 7s
2:	learn: 0.8787238	total: 415ms	remaining: 2m 17s
3:	learn: 0.8767797	total: 564ms	remaining: 2m 20s
4:	learn: 0.8751148	total: 655ms	remaining: 2m 10s
5:	learn: 0.8734477	total: 770ms	remaining: 2m 7s
6:	learn: 0.8720372	total: 869ms	remaining: 2m 3s
7:	learn: 0.8706866	total: 1s	remaining: 2m 4s
8:	learn: 0.8695030	total: 1.12s	remaining: 2m 3s
9:	learn: 0.8683646	total: 1.21s	remaining: 1m 59s
10:	learn: 0.8674941	total: 1.33s	remaining: 2m
11:	learn: 0.8666516	total: 1.45s	remaining: 1m 59s
12:	learn: 0.8658779	total: 1.57s	remaining: 1m 59s
13:	learn: 0.8650691	total: 1.68s	remaining: 1m 58s
14:	learn: 0.8643753	total: 1.81s	remaining: 1m 58s
15:	learn: 0.8636825	total: 1.94s	remaining: 1m 59s
16:	learn: 0.8631119	total: 2.03s	remaining: 1m 57s
17:	learn: 0.8625835	total: 2.17s	remaining: 1m 58s
18:	learn: 0.8621285	total: 2.31s	remaining: 1m 59s
19:	

160:	learn: 0.8442475	total: 21s	remaining: 1m 49s
161:	learn: 0.8442011	total: 21.1s	remaining: 1m 49s
162:	learn: 0.8441479	total: 21.3s	remaining: 1m 49s
163:	learn: 0.8441207	total: 21.4s	remaining: 1m 48s
164:	learn: 0.8440734	total: 21.5s	remaining: 1m 48s
165:	learn: 0.8440173	total: 21.6s	remaining: 1m 48s
166:	learn: 0.8439455	total: 21.7s	remaining: 1m 48s
167:	learn: 0.8438895	total: 21.9s	remaining: 1m 48s
168:	learn: 0.8438367	total: 22s	remaining: 1m 48s
169:	learn: 0.8437890	total: 22.1s	remaining: 1m 48s
170:	learn: 0.8437568	total: 22.3s	remaining: 1m 48s
171:	learn: 0.8437068	total: 22.4s	remaining: 1m 47s
172:	learn: 0.8436633	total: 22.5s	remaining: 1m 47s
173:	learn: 0.8436110	total: 22.7s	remaining: 1m 47s
174:	learn: 0.8435467	total: 22.8s	remaining: 1m 47s
175:	learn: 0.8434949	total: 23s	remaining: 1m 47s
176:	learn: 0.8434372	total: 23.1s	remaining: 1m 47s
177:	learn: 0.8433798	total: 23.2s	remaining: 1m 47s
178:	learn: 0.8433383	total: 23.4s	remaining: 1m 47s

316:	learn: 0.8386828	total: 41.6s	remaining: 1m 29s
317:	learn: 0.8386657	total: 41.7s	remaining: 1m 29s
318:	learn: 0.8386605	total: 41.8s	remaining: 1m 29s
319:	learn: 0.8386280	total: 41.9s	remaining: 1m 29s
320:	learn: 0.8385909	total: 42.1s	remaining: 1m 28s
321:	learn: 0.8385614	total: 42.2s	remaining: 1m 28s
322:	learn: 0.8385219	total: 42.3s	remaining: 1m 28s
323:	learn: 0.8384843	total: 42.5s	remaining: 1m 28s
324:	learn: 0.8384515	total: 42.6s	remaining: 1m 28s
325:	learn: 0.8384184	total: 42.7s	remaining: 1m 28s
326:	learn: 0.8383737	total: 42.9s	remaining: 1m 28s
327:	learn: 0.8383699	total: 43s	remaining: 1m 28s
328:	learn: 0.8383618	total: 43.1s	remaining: 1m 27s
329:	learn: 0.8383336	total: 43.3s	remaining: 1m 27s
330:	learn: 0.8382983	total: 43.4s	remaining: 1m 27s
331:	learn: 0.8382750	total: 43.5s	remaining: 1m 27s
332:	learn: 0.8382379	total: 43.6s	remaining: 1m 27s
333:	learn: 0.8382164	total: 43.8s	remaining: 1m 27s
334:	learn: 0.8381779	total: 43.9s	remaining: 1m

472:	learn: 0.8349603	total: 1m 1s	remaining: 1m 8s
473:	learn: 0.8349264	total: 1m 1s	remaining: 1m 8s
474:	learn: 0.8348977	total: 1m 2s	remaining: 1m 8s
475:	learn: 0.8348826	total: 1m 2s	remaining: 1m 8s
476:	learn: 0.8348773	total: 1m 2s	remaining: 1m 8s
477:	learn: 0.8348532	total: 1m 2s	remaining: 1m 8s
478:	learn: 0.8348367	total: 1m 2s	remaining: 1m 7s
479:	learn: 0.8348159	total: 1m 2s	remaining: 1m 7s
480:	learn: 0.8348023	total: 1m 2s	remaining: 1m 7s
481:	learn: 0.8347808	total: 1m 2s	remaining: 1m 7s
482:	learn: 0.8347587	total: 1m 3s	remaining: 1m 7s
483:	learn: 0.8347293	total: 1m 3s	remaining: 1m 7s
484:	learn: 0.8347029	total: 1m 3s	remaining: 1m 7s
485:	learn: 0.8346878	total: 1m 3s	remaining: 1m 7s
486:	learn: 0.8346712	total: 1m 3s	remaining: 1m 6s
487:	learn: 0.8346701	total: 1m 3s	remaining: 1m 6s
488:	learn: 0.8346428	total: 1m 3s	remaining: 1m 6s
489:	learn: 0.8346179	total: 1m 3s	remaining: 1m 6s
490:	learn: 0.8345880	total: 1m 4s	remaining: 1m 6s
491:	learn: 

629:	learn: 0.8318045	total: 1m 22s	remaining: 48.2s
630:	learn: 0.8317741	total: 1m 22s	remaining: 48.1s
631:	learn: 0.8317516	total: 1m 22s	remaining: 47.9s
632:	learn: 0.8317212	total: 1m 22s	remaining: 47.8s
633:	learn: 0.8316954	total: 1m 22s	remaining: 47.7s
634:	learn: 0.8316805	total: 1m 22s	remaining: 47.5s
635:	learn: 0.8316634	total: 1m 22s	remaining: 47.4s
636:	learn: 0.8316437	total: 1m 22s	remaining: 47.3s
637:	learn: 0.8316304	total: 1m 23s	remaining: 47.1s
638:	learn: 0.8316011	total: 1m 23s	remaining: 47s
639:	learn: 0.8315837	total: 1m 23s	remaining: 46.9s
640:	learn: 0.8315701	total: 1m 23s	remaining: 46.7s
641:	learn: 0.8315451	total: 1m 23s	remaining: 46.6s
642:	learn: 0.8315172	total: 1m 23s	remaining: 46.5s
643:	learn: 0.8314998	total: 1m 23s	remaining: 46.3s
644:	learn: 0.8314767	total: 1m 23s	remaining: 46.2s
645:	learn: 0.8314731	total: 1m 24s	remaining: 46.1s
646:	learn: 0.8314456	total: 1m 24s	remaining: 45.9s
647:	learn: 0.8314415	total: 1m 24s	remaining: 4

785:	learn: 0.8286228	total: 1m 42s	remaining: 27.8s
786:	learn: 0.8286059	total: 1m 42s	remaining: 27.7s
787:	learn: 0.8285916	total: 1m 42s	remaining: 27.5s
788:	learn: 0.8285720	total: 1m 42s	remaining: 27.4s
789:	learn: 0.8285652	total: 1m 42s	remaining: 27.3s
790:	learn: 0.8285397	total: 1m 42s	remaining: 27.2s
791:	learn: 0.8285133	total: 1m 42s	remaining: 27s
792:	learn: 0.8285068	total: 1m 43s	remaining: 26.9s
793:	learn: 0.8284866	total: 1m 43s	remaining: 26.8s
794:	learn: 0.8284752	total: 1m 43s	remaining: 26.6s
795:	learn: 0.8284679	total: 1m 43s	remaining: 26.5s
796:	learn: 0.8284576	total: 1m 43s	remaining: 26.4s
797:	learn: 0.8284338	total: 1m 43s	remaining: 26.2s
798:	learn: 0.8284178	total: 1m 43s	remaining: 26.1s
799:	learn: 0.8283935	total: 1m 43s	remaining: 26s
800:	learn: 0.8283774	total: 1m 44s	remaining: 25.9s
801:	learn: 0.8283563	total: 1m 44s	remaining: 25.7s
802:	learn: 0.8283292	total: 1m 44s	remaining: 25.6s
803:	learn: 0.8283205	total: 1m 44s	remaining: 25.

942:	learn: 0.8257971	total: 2m 2s	remaining: 7.41s
943:	learn: 0.8257852	total: 2m 2s	remaining: 7.28s
944:	learn: 0.8257674	total: 2m 2s	remaining: 7.15s
945:	learn: 0.8257450	total: 2m 2s	remaining: 7.02s
946:	learn: 0.8257196	total: 2m 3s	remaining: 6.89s
947:	learn: 0.8256909	total: 2m 3s	remaining: 6.76s
948:	learn: 0.8256638	total: 2m 3s	remaining: 6.63s
949:	learn: 0.8256421	total: 2m 3s	remaining: 6.5s
950:	learn: 0.8256210	total: 2m 3s	remaining: 6.37s
951:	learn: 0.8255996	total: 2m 3s	remaining: 6.24s
952:	learn: 0.8255935	total: 2m 3s	remaining: 6.11s
953:	learn: 0.8255719	total: 2m 3s	remaining: 5.98s
954:	learn: 0.8255492	total: 2m 4s	remaining: 5.85s
955:	learn: 0.8255417	total: 2m 4s	remaining: 5.72s
956:	learn: 0.8255139	total: 2m 4s	remaining: 5.59s
957:	learn: 0.8254964	total: 2m 4s	remaining: 5.46s
958:	learn: 0.8254678	total: 2m 4s	remaining: 5.33s
959:	learn: 0.8254368	total: 2m 4s	remaining: 5.2s
960:	learn: 0.8254272	total: 2m 4s	remaining: 5.07s
961:	learn: 0.

,id,target
0,0,7.710960
1,5,7.732050
2,15,7.729660
3,16,7.527546
4,17,7.285527


In [20]:
submission.to_csv('submission_catboost_default.csv', index=False)

# Train a random forest

In [53]:
x_train_oh = pd.get_dummies(x_train)
test_oh = pd.get_dummies(test)

In [54]:
test_oh.shape

(200000, 70)

In [55]:
x_train_oh.shape

(300000, 70)

In [56]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=600, n_jobs=-2, random_state=42, min_samples_leaf=0.0001, verbose=1)

In [57]:
scores = cross_validate(rf, X=x_train_oh, y=y_train, cv=5, return_train_score = True,
                         scoring='neg_root_mean_squared_error', verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-2)]: Done 186 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-2)]: Done 436 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-2)]: Done 600 out of 600 | elapsed: 12.2min finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    0.4s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:    0.8s
[Parallel(n_jobs=7)]: Done 600 out of 600 | elapsed:    1.1s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    1.4s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed

In [58]:
scores

{'fit_time': array([735.32043934, 736.87023497, 746.3355546 , 737.5243113 ,
        741.24415064]),
 'score_time': array([1.15778542, 1.10660458, 1.13768029, 1.13884997, 1.13567924]),
 'test_score': array([-0.85180736, -0.85246847, -0.85403798, -0.85382717, -0.85278999]),
 'train_score': array([-0.74666176, -0.74648601, -0.74611985, -0.74625115, -0.74675153])}

In [59]:
scores['test_score'].mean()

-0.8529861930591363

# Lightgbm parameter optimization

In [51]:
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV

In [52]:
from scipy.stats import uniform
from scipy.stats import expon
from sklearn.utils.fixes import loguniform

In [55]:
params = {'n_estimators':[10, 20, 50, 100, 200, 500, 1000, 2000],
          'boosting_type':['gbdt', 'dart'],
          'num_leaves':[10, 31, 50, 100, 200]}

In [56]:
lgb = LGBMRegressor(n_jobs=-2, verbose=1)

In [57]:
cv = RandomizedSearchCV(lgb, params, return_train_score=True, verbose=100, scoring='neg_root_mean_squared_error')

In [ ]:
cv.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] num_leaves=100, n_estimators=2000, boosting_type=gbdt ...........
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014334 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3629
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 24
[LightGBM] [Info] Start training from score 7.456400
[CV]  num_leaves=100, n_estimators=2000, boosting_type=gbdt, score=(train=-0.506, test=-0.862), total=  15.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.9s remaining:    0.0s
[CV] num_leaves=100, n_estimators=2000, boosting_type=gbdt ...........
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014319 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Inf

[CV]  num_leaves=10, n_estimators=50, boosting_type=dart, score=(train=-1.110, test=-1.111), total=   0.8s
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:  1.8min remaining:    0.0s
[CV] num_leaves=10, n_estimators=50, boosting_type=dart ..............
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006149 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3630
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 24
[LightGBM] [Info] Start training from score 7.456370
[CV]  num_leaves=10, n_estimators=50, boosting_type=dart, score=(train=-1.111, test=-1.111), total=   0.8s
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:  1.8min remaining:    0.0s
[CV] num_leaves=10, n_estimators=50, boosting_type=dart ..............
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead 

In [45]:
cv.best_params_

{'num_leaves': 10, 'n_estimators': 4000, 'boosting_type': 'dart'}

In [48]:
cv.cv_results_

{'mean_fit_time': array([  6.20615635,   1.02770467,  11.75101247,   0.85226884,
          1.14812436,   1.26146255,  38.39643149, 268.24906111,
          0.5486321 ,   0.46301999]),
 'std_fit_time': array([0.10216764, 0.01163098, 0.27602386, 0.02625177, 0.00599337,
        0.00996432, 0.43797059, 1.09087225, 0.00126018, 0.00764438]),
 'mean_score_time': array([0.46361566, 0.05400543, 0.28429856, 0.05977254, 0.05956674,
        0.09815326, 0.51990366, 1.878581  , 0.03519893, 0.02887855]),
 'std_score_time': array([0.00321334, 0.00060293, 0.00281288, 0.00078433, 0.00034045,
        0.00074121, 0.00496131, 0.01318892, 0.00029734, 0.00077851]),
 'param_num_leaves': masked_array(data=[50, 31, 200, 50, 50, 50, 100, 10, 50, 100],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1000, 50, 200, 50, 50, 100, 500, 4000, 20, 10],
             